### 豆瓣评分的预测

在这个项目中，我们要预测一部电影的评分，这个问题实际上就是一个分类问题。给定的输入为一段文本，输出为具体的评分。 在这个项目中，我们需要做：
- 文本的预处理，如停用词的过滤，低频词的过滤，特殊符号的过滤等
- 文本转化成向量，将使用三种方式，分别为tf-idf, word2vec以及BERT向量。 
- 训练逻辑回归和朴素贝叶斯模型，并做交叉验证
- 评估模型的准确率

在具体标记为``TODO``的部分填写相应的代码。 

In [2]:
#from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#pip install bert_embedding

In [ ]:
pip install mxnet-cu100

In [5]:

#导入数据处理的基础包
import numpy as np
import pandas as pd

#导入用于计数的包
from collections import Counter

#导入tf-idf相关的包
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer

#导入模型评估的包
from sklearn import metrics

#导入与word2vec相关的包
from gensim.models import KeyedVectors

#导入与bert embedding相关的包，关于mxnet包下载的注意事项参考实验手册
from bert_embedding import BertEmbedding
import mxnet
from mxnet import gpu
import jieba
#包tqdm是用来对可迭代对象执行时生成一个进度条用以监视程序运行过程
from tqdm import tqdm

#导入其他一些功能包
import requests
import os

### 1. 读取数据并做文本的处理
你需要完成以下几步操作：
- 去掉无用的字符如！&，可自行定义
- 中文分词
- 去掉低频词

In [ ]:
#读取数据
#data = pd.read_csv('/content/drive/MyDrive/Greedy/project1-douban/data/DMSC.csv')
data = pd.read_csv('data/DMSC.csv')
#观察数据格式
data.head()

In [ ]:
data['34'，]

In [ ]:
#输出数据的一些相关信息
data.info()

In [ ]:
#只保留数据中我们需要的两列：Comment列和Star列
data = data[['Comment','Star']]
#观察新的数据的格式
data.head()

In [ ]:
# 这里的star代表具体的评分。但在这个项目中，我们要预测的是正面还是负面。我们把评分为1和2的看作是负面，把评分为3，4，5的作为正面
data['Star']=(data.Star/3).astype(int)
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国。,1
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1


#### 任务1： 去掉一些无用的字符

In [ ]:
# TODO1: 去掉一些无用的字符，自行定一个字符几何，并从文本中去掉
#    your to do 
import re 
def clear_character(sentence):
    pattern1='[a-zA-Z0-9]'
    pattern2 = re.compile(u'[^\s1234567890:：' + '\u4e00-\u9fa5]+')
    pattern3='[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+'
    line1=re.sub(pattern1,'',sentence)   #去除英文字母和数字
    line2=re.sub(pattern2,'',line1)   #去除表情和其他字符
    line3=re.sub(pattern3,'',line2)   #去除去掉残留的冒号及其它符号
    new_sentence=''.join(line3.split()) #去除空白
    return new_sentence

tqdm.pandas(desc='apply')
data['Comment'] = data['Comment'].progress_apply(clear_character)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
apply: 100%|██████████| 212506/212506 [00:01<00:00, 127828.88it/s]


In [ ]:
data.head()

#### 任务2：使用结巴分词对文本做分词

In [ ]:
# TODO2: 导入中文分词包jieba, 并用jieba对原始文本做分词
def comment_cut(content):
    # TODO: 使用结巴完成对每一个comment的分词
    document_cut = jieba.cut(content)
    word = []
    for i in document_cut:
      word.append(i)
    
    return word

# 输出进度条
tqdm.pandas(desc='apply')
data['comment_processed'] = data['Comment'].progress_apply(comment_cut)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
apply:   0%|          | 0/212506 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.745 seconds.
Prefix dict has been built successfully.
apply: 100%|██████████| 212506/212506 [00:31<00:00, 6776.18it/s]


In [ ]:
# 观察新的数据的格式
data.head()

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,"[连, 奥创, 都, 知道, 整容, 要, 去, 韩国]"
1,一个没有黑暗面的人不值得信任第二部剥去冗长的铺垫开场即高潮一直到结束会有人觉得只剩动作特技不...,1,"[一个, 没有, 黑暗面, 的, 人, 不, 值得, 信任, 第二部, 剥去, 冗长, 的,..."
2,奥创弱爆了弱爆了弱爆了啊,0,"[奥创, 弱, 爆, 了, 弱, 爆, 了, 弱, 爆, 了, 啊]"
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战又...,1,"[与, 第一集, 不同, 承上启下, 阴郁, 严肃, 但, 也, 不会, 不, 好看, 啊,..."
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹,1,"[看毕, 我, 激动, 地, 对, 友人, 说, 等等, 奥创, 要, 来, 毁灭, 台北,..."


#### 任务3：设定停用词并去掉停用词

In [ ]:
# TODO3: 设定停用词并从文本中去掉停用词

# 下载中文停用词表至data/stopWord.json中，下载地址:https://github.com/goto456/stopwords/
#if not os.path.exists('/content/drive/MyDrive/Greedy/project1-douban/data/stopWord.json'):
#    stopWord = requests.get("https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt")
#    with open("data/stopWord.json", "wb") as f:
#         f.write(stopWord.content)

# 读取下载的停用词表，并保存在列表中
#with open("data/stopWord.json","r") as f:
with open("/content/drive/MyDrive/Greedy/project1-douban/data/stopWord.json","r") as f:    
    stopWords = f.read().split("\n")  
    
    
# 去除停用词
def rm_stop_word(wordList):
    # your code, remove stop words
    tokenized_com = [w for w in wordList if not w in stopWords]
    return tokenized_com

#这行代码中.progress_apply()函数的作用等同于.apply()函数的作用，只是写成.progress_apply()函数才能被tqdm包监控从而输出进度条。
tqdm.pandas(desc='apply')
data['comment_processed'] = data['comment_processed'].progress_apply(rm_stop_word)

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
apply: 100%|██████████| 212506/212506 [00:43<00:00, 4850.83it/s]


In [ ]:
# 观察新的数据的格式
data.head()

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,"[奥创, 知道, 整容, 韩国]"
1,一个没有黑暗面的人不值得信任第二部剥去冗长的铺垫开场即高潮一直到结束会有人觉得只剩动作特技不...,1,"[一个, 没有, 黑暗面, 值得, 信任, 第二部, 剥去, 冗长, 铺垫, 开场, 高潮,..."
2,奥创弱爆了弱爆了弱爆了啊,0,"[奥创, 弱, 爆, 弱, 爆, 弱, 爆]"
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战又...,1,"[第一集, 不同, 承上启下, 阴郁, 严肃, 不会, 好看, 本来, 喜欢, 漫威, 电影..."
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹,1,"[看毕, 激动, 友人, 说, 奥创, 毁灭, 台北, 厚, 拍了拍, 肩膀, 没事, 反正..."


#### 任务4：去掉低频词，出现次数少于10次的词去掉

In [ ]:
# TODO4: 去除低频词, 去掉词频小于10的单词，
from collections import Counter
#统计词频
word_list = Counter()

for i in tqdm(range(len(data['comment_processed']))):
  tmp = Counter(data['comment_processed'][i])
  word_list+=tmp

#data['comment_processed'] = 

100%|██████████| 212506/212506 [11:16<00:00, 314.22it/s]


In [ ]:
#收集低频词
low_fre = []
for i in tqdm(range(len(data['comment_processed']))):
  for term in data['comment_processed'][i]:
    if word_list[term] < 10:
      low_fre.append(term)
len(low_fre)

100%|██████████| 212506/212506 [00:01<00:00, 193647.97it/s]


171880

In [ ]:
#去除低频词 并把结果存放在data['comment_processed']里
def rm_low_fre_word(wordList):

    tokenized_com = [w for w in wordList if not w in low_fre]
    return tokenized_com

tqdm.pandas(desc='apply')
data['comment_processed'] = data['comment_processed'].progress_apply(rm_low_fre_word)

/Users/bin/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
apply: 100%|██████████| 212506/212506 [6:19:29<00:00,  9.33it/s]   


In [ ]:
# 观察新的数据的格式
data.head()

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,奥创 知道 整容 韩国
1,一个没有黑暗面的人不值得信任第二部剥去冗长的铺垫开场即高潮一直到结束会有人觉得只剩动作特技不...,1,一个 没有 黑暗面 值得 信任 第二部 冗长 铺垫 开场 高潮 一直 结束 会 有人 觉得 ...
2,奥创弱爆了弱爆了弱爆了啊,0,奥创 弱 爆 弱 爆 弱 爆
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战又...,1,第一集 不同 承上启下 阴郁 严肃 不会 好看 本来 喜欢 漫威 电影 场面 更加 宏大 团...
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹,1,激动 友人 说 奥创 毁灭 台北 厚 肩膀 没事 反正 买 两份 旅行 惹


In [ ]:
def list_to_str(sentence):
  outstr = ''
  for word in sentence:
      outstr += word
      outstr += " "
  return outstr

In [ ]:
tqdm.pandas(desc='apply')
data['comment_processed'] = data['comment_processed'].progress_apply(list_to_str)

/Users/bin/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
apply: 100%|██████████| 212506/212506 [00:00<00:00, 548772.23it/s]


In [ ]:
data.to_csv('data/DMSC_processed.csv')

In [142]:
#data_processed = pd.read_csv('/content/drive/MyDrive/Greedy/project1-douban/data/DMSC_processed.csv')
data_processed = pd.read_csv('data/DMSC_processed.csv')
data_without_NaN = data_processed.dropna(axis=0)
data_without_NaN.head(50)

,Unnamed: 0,Comment,Star,comment_processed
0,0,连奥创都知道整容要去韩国,1,奥创 知道 整容 韩国
1,1,一个没有黑暗面的人不值得信任第二部剥去冗长的铺垫开场即高潮一直到结束会有人觉得只剩动作特技不...,1,一个 没有 黑暗面 值得 信任 第二部 冗长 铺垫 开场 高潮 一直 结束 会 有人 觉得 ...
2,2,奥创弱爆了弱爆了弱爆了啊,0,奥创 弱 爆 弱 爆 弱 爆
3,3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战又...,1,第一集 不同 承上启下 阴郁 严肃 不会 好看 本来 喜欢 漫威 电影 场面 更加 宏大 团...
4,4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹,1,激动 友人 说 奥创 毁灭 台北 厚 肩膀 没事 反正 买 两份 旅行 惹
5,5,绝逼不质疑尾灯的导演和编剧水平,1,绝逼 质疑 尾灯 导演 编剧 水平
6,6,睡着次睡着两次,0,睡着 次 睡着 两次
7,7,谁再喊我看这种电影我和谁急实在是接受无能,0,喊 这种 电影 急 实在 接受 无能
8,8,超愉悦以及超满足在历经了第一阶段比漫画更普世的设定融合之后发展到居然出现了不少传统科幻的哲思...,1,超 愉悦 超 满足 历经 漫画 设定 融合 之后 发展 居然 出现 不少 传统 科幻 尾灯 ...
9,9,观影过程中耳边一直有一种突突突突突的声音我还感慨电影为了让奥创给观众带来紧张感声音上真是下了...,1,观影 过程 中 耳边 一直 一种 突突突 声音 感慨 电影 奥创 观众 带来 紧张感 声音 ...


### 2. 把文本分为训练集和测试集
选择语料库中的20%作为测试数据，剩下的作为训练数据

In [143]:
# TODO5: 把数据分为训练集和测试集. comments_train（list)保存用于训练的文本，comments_test(list)保存用于测试的文本。 y_train, y_test是对应的标签（0、1）
test_ratio = 0.2
train_num = int((1-test_ratio)*len(data_without_NaN['comment_processed']))

comment_list = []
for line in data_without_NaN['Comment']:
  comment_list.append([line])

tokenized_comm_list = []
for line in data_without_NaN['comment_processed']:
  tokenized_comm_list.append([line])

star_list = []
for line in data_without_NaN['Star']:
  star_list.append(line)
len(tokenized_comm_list)

# Label
y_train, y_test  = star_list[:train_num], star_list[train_num:]

# Tfidf训练集和测试集
tfidf_train = []
tfidf_test = []

for i in range(train_num):
  tfidf_train = tfidf_train + tokenized_comm_list[i]
for i in range(train_num ,len(tokenized_comm_list)):
  tfidf_test = tfidf_test + tokenized_comm_list[i]

# Word2Vec训练集和测试集
word2veclist_train = []
word2veclist_test = []

for i in range(train_num):
  word2veclist_train.append(tokenized_comm_list[i])
for i in range(train_num ,len(tokenized_comm_list)):
  word2veclist_test.append(tokenized_comm_list[i])

# Bert-Embedding训练集和测试集
bert_comments_train, bert_comments_test = comment_list[:train_num], comment_list[train_num:]


### 3. 把文本转换成向量的形式

在这个部分我们会采用三种不同的方式:
- 使用tf-idf向量
- 使用word2vec
- 使用bert向量

转换成向量之后，我们接着做模型的训练

#### 任务6：把文本转换成tf-idf向量

In [149]:
# TODO6: 把训练文本和测试文本转换成tf-idf向量。使用sklearn的feature_extraction.text.TfidfTransformer模块
#    请留意fit_transform和transform之间的区别。 常见的错误是在训练集和测试集上都使用 fit_transform，需要避免！ 
#    另外，可以留意一下结果是否为稀疏矩阵
from  sklearn.feature_extraction.text  import  CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer#TfidfVectorizer类可以帮助完成向量化、TF-IDF和标准化三步。当然，还可以帮我们处理停用词。

vectorizer = CountVectorizer()
transformer = TfidfTransformer()

tfidf_tr = transformer.fit_transform(vectorizer.fit_transform(tfidf_train))

tfidf_te = transformer.transform(vectorizer.transform(tfidf_test))

print (tfidf_tr.shape, tfidf_te.shape)

(165972, 14235) (41494, 14235)


#### 任务7：把文本转换成word2vec向量

In [146]:
# 由于训练出一个高效的word2vec词向量往往需要非常大的语料库与计算资源，所以我们通常不自己训练Wordvec词向量，而直接使用网上开源的已训练好的词向量。
# data/sgns.zhihu.word是从https://github.com/Embedding/Chinese-Word-Vectors下载到的预训练好的中文词向量文件
# 使用KeyedVectors.load_word2vec_format()函数加载预训练好的词向量文件
model = KeyedVectors.load_word2vec_format('data/sgns.zhihu.word')

In [10]:
#预训练词向量使用举例
model['今天'].shape

(300,)

In [11]:
vocabulary = model.vocab

In [147]:
# TODO7: 对于每个句子，生成句子的向量。具体的做法是：包含在句子中的所有单词的向量做平均。

def sent2vec(sentence_list):
    
    words = sentence_list[0].strip().split(' ')
    sent_vec = np.zeros([300,])
    for item in words:
        if item not in vocabulary:
            continue
        else:
            sent_vec = sent_vec + model[item]
    sent_vec = sent_vec/len(words)
    
    return sent_vec
def comm_list2vec(comm_list):
    
    result = []
    
    for i in tqdm(range(len(comm_list))): 
        result.append(sent2vec(comm_list[i]))
    
    return np.array(result)
    

In [148]:
word2vec_train = comm_list2vec(word2veclist_train)

word2vec_test = comm_list2vec(word2veclist_test)

print (word2vec_train.shape, word2vec_test.shape)

100%|██████████| 41494/41494 [00:01<00:00, 36562.55it/s]

(165972, 300) (41494, 300)


#### 任务8：把文本转换成bert向量

In [15]:
# 导入gpu版本的bert embedding预训练的模型。
# 若没有gpu，则ctx可使用其默认值cpu(0)。但使用cpu会使程序运行的时间变得非常慢
# 若之前没有下载过bert embedding预训练的模型，执行此句时会花费一些时间来下载预训练的模型
ctx = mxnet.gpu()

embedding = BertEmbedding(ctx = ctx, model='bert_12_768_12', dataset_name='wiki_cn')

/usr/local/lib/python3.7/dist-packages/gluonnlp/model/bert.py:693: UserWarning: wiki_cn/wiki_multilingual will be deprecated. Please use wiki_cn_cased/wiki_multilingual_uncased instead.
  warnings.warn('wiki_cn/wiki_multilingual will be deprecated.'


Vocab file is not found. Downloading.


In [16]:
def ger_sen_vec(word_vec):

  vec_array = np.zeros([768,])
  for i in range(1, len(word_vec[0][1])-1):
    vec_array = vec_array + np.array(word_vec[0][1][i])
  new_vec = vec_array/(len(word_vec[0][1]) - 2)
  return new_vec

def get_bert_vec (sentences):
  bert_train = []
  for i in tqdm(range(len(sentences))):
    restu1 = embedding(sentences[i])
    bert_vec = ger_sen_vec(restu1)
    bert_train.append(bert_vec)
  return np.array(bert_train)

In [17]:
bert_train = get_bert_vec(bert_comments_train)
bert_test = get_bert_vec(bert_comments_test)

100%|██████████| 41494/41494 [28:59<00:00, 23.86it/s]


In [18]:
print (bert_train.shape, bert_test.shape)

(165972, 768) (41494, 768)


In [19]:
print (tfidf_tr.shape, tfidf_te.shape)
print (word2vec_train.shape, word2vec_test.shape)
print (bert_train.shape, bert_test.shape)

(165972, 14235) (41494, 14235)
(165972, 300) (41494, 300)
(165972, 768) (41494, 768)


### 4. 训练模型以及评估
对如上三种不同的向量表示法，分别训练逻辑回归模型，需要做：
- 搭建模型
- 训练模型（并做交叉验证）
- 输出最好的结果

In [150]:
# 导入逻辑回归的包
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

#### 任务9：使用tf-idf，并结合逻辑回归训练模型

In [151]:
# TODO9: 使用tf-idf + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数

tfidf_lr = LR(penalty="l2",solver = 'sag',max_iter = 500)

param = {
    'C' : [0.09,0.1,0.15,0.2,0.4,0.5,0.7]
}
grid = GridSearchCV(estimator = tfidf_lr, 
                    param_grid = param, 
                    cv = 5
                   )

grid.fit(tfidf_tr,y_train)
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)

{'C': 0.1}
0.8213131030500079
LogisticRegression(C=0.1, max_iter=500, solver='sag')


In [152]:
tfidf_lr = LR(C = 0.1, max_iter = 500, solver = 'sag')
tfidf_lr.fit(tfidf_tr,y_train)
tf_idf_y_pred = tfidf_lr.predict(tfidf_te)
print('TF-IDF LR test accuracy %s' % metrics.accuracy_score(y_test, tf_idf_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('TF-IDF LR test F1_score %s' % metrics.f1_score(y_test, tf_idf_y_pred,average="macro"))

TF-IDF LR test accuracy 0.9375331373210585
TF-IDF LR test F1_score 0.6337616919812451


#### 任务10：使用word2vec，并结合逻辑回归训练模型

In [153]:
# TODO10: 使用word2vec + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数

word2vec_lr = LR(penalty="l2",solver = 'sag',max_iter = 500)

param = {
    'C' : [0.01,0.03,0.06,0.08,0.09,0.1,0.15,0.2,0.4,0.5]
}
word2vec_grid = GridSearchCV(estimator = tfidf_lr, 
                    param_grid = param, 
                    cv = 5 
                   )

word2vec_grid.fit(word2vec_train,y_train)
print(word2vec_grid.best_params_)
print(word2vec_grid.best_score_)
print(word2vec_grid.best_estimator_)


{'C': 0.03}
0.8232773579191386
LogisticRegression(C=0.03, max_iter=500, solver='sag')


In [154]:
print('Word2vec LR test accuracy %s' % metrics.accuracy_score(y_test, word2vec_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('Word2vec LR test F1_score %s' % metrics.f1_score(y_test, word2vec_y_pred,average="macro"))

Word2vec LR test accuracy 0.923386513712826
Word2vec LR test F1_score 0.6444193008059403


#### 任务11：使用bert，并结合逻辑回归训练模型

In [23]:
# TODO11: 使用bert + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数
bert_lr = LR(penalty="l2",max_iter = 500,solver = 'sag')

param = {
    'C' : [0.09,0.1,0.2,0.4,0.7]
}
bert_grid = GridSearchCV(estimator = bert_lr, 
                    param_grid = param, 
                    cv = 5 
                   )

bert_grid.fit(bert_train,y_train)
print(bert_grid.best_params_)
print(bert_grid.best_score_)
print(bert_grid.best_estimator_)

{'C': 0.09}
0.814830182662399
LogisticRegression(C=0.09, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)


In [24]:
bert_lr = LR(penalty="l2",solver="sag",C=0.09,max_iter=500)
bert_lr.fit(bert_train,y_train)
 
bert_y_pred = bert_lr.predict(bert_test)

print('Bert LR test accuracy %s' % metrics.accuracy_score(y_test, bert_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('Bert LR test F1_score %s' % metrics.f1_score(y_test, bert_y_pred,average="macro"))

Bert LR test accuracy 0.9207355280281486
Bert LR test F1_score 0.6131981416847541


#### 任务12：对于以上结果请做一下简单的总结，按照1，2，3，4提取几个关键点，包括：
- 结果说明什么问题？
- 接下来如何提高？

1.jieba分词有时候会将一些不该分开的词汇分开，例如它将“不值得”分成“不”，“值得”，而后“不”被当作停用词去除，影响了整体的语义。分词词库还得根据应用场景做对应修改。 

2.在word2vec时才发现有空文本存在，应该在处理数据时就该发现处理。

3.通过逻辑回归建模三种word embedding的准确率差不多，在测试集准确率都能达到92%，但是f1不是很高，说明2分类sample不均匀。

4.分词结果有些词汇不在word2vec的词库中，我选择了直接将该词跳过，不做word3vec，有待改进。

5.可以使用更复杂的模型分类，如神经网络来提高准确率。
